# Imports

In [1]:
# 필요한 모듈 임포트
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from PIL import ImageGrab

# 웹캠
import cv2
cap=cv2.VideoCapture(0)

## Env setup

In [2]:
# 저장할 path 설정
sys.path.append("..")

from utils import label_map_util
from utils import visualization_utils as vis_util

# Model preparation 

In [3]:
# 다운로드할 모델
MODEL_NAME = 'ssd_mobilenet_v1_coco_2017_11_17'
MODEL_FILE = MODEL_NAME + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

# webcam->image: Object Detection 할 때 사용
# pb: 모델 구조와 가중치 값이 합쳐진 파일(binary). 학습 시에 그렸던 그래프 구조가 정의되어 있음. 용량 작음
# ckpt: 학습된 모델의 구조를 제외한 변수들을 담고 있는 파일->재학습 가능. 모델의 가중치만 담고있는 파일
PATH_TO_CKPT = MODEL_NAME + '/frozen_inference_graph.pb'

# 이미지에 라벨 붙이기
# pbtxt: text파일. 사람이 읽을 수 있는 형식->디버깅/편집에 적합. 용량 큼
PATH_TO_LABELS = os.path.join('data', 'mscoco_label_map.pbtxt')

# 클래스 개수
NUM_CLASSES = 90

## Download Model

In [4]:
#인터넷 리소스 다운로드
opener = urllib.request.URLopener()
opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)
tar_file = tarfile.open(MODEL_FILE)
for file in tar_file.getmembers():
  file_name = os.path.basename(file.name)
  if 'frozen_inference_graph.pb' in file_name:
    tar_file.extract(file, os.getcwd())

## Load a (frozen) Tensorflow model into memory.

In [5]:
detection_graph = tf.Graph()
with detection_graph.as_default(): #텐서플로우 계산은 텐서플로우 그래프로 표현됨
  od_graph_def = tf.compat.v1.GraphDef() #tf 2.3.0
  with tf.compat.v2.io.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:# protobuf 형식으로 정의된 내용을 string(그래프에 대한 정보)으로 파싱
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

## Loading label map
Label maps map indices to category names, so that when our convolution network predicts `5`, we know that this corresponds to `airplane`.  Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine

In [6]:
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

In [7]:
flag=True

with detection_graph.as_default():
  with tf.compat.v1.Session(graph=detection_graph) as sess:
    
    while (True):
        
        ret, image_np=cap.read() 

         # 차원 확장: 이미지가 shape가질 것이라 기대
        image_np_expanded = np.expand_dims(image_np, axis=0)
        # 입력/출력 텐서 정의
        image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
        # 감지한 object box형성
        # by regression
        # (x1,y1,x2,y2)
        # grab_screen?
        boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
        # 확률 나타냄(label과 함께 표시)
        scores = detection_graph.get_tensor_by_name('detection_scores:0')
        classes = detection_graph.get_tensor_by_name('detection_classes:0')
        num_detections = detection_graph.get_tensor_by_name('num_detections:0')
       
         
          # Actual detection.
        (boxes, scores, classes, num) = sess.run(
              [boxes, scores, classes, num_detections],
              feed_dict={image_tensor: image_np_expanded})
          # Visualization of the results of a detection.
        coords=vis_util.visualize_boxes_and_labels_on_image_array(
              image_np,
              np.squeeze(boxes),
              np.squeeze(classes).astype(np.int32),
              np.squeeze(scores),
              category_index,
              use_normalized_coordinates=True,
              line_thickness=8)
        

                
                #img=ImageGrab.grab((x1,y1,x2,y2))
                #image.show()
            #print(coords[0][0])
       # print(np.shape(coords))
        #print(coords[0][1])
        #print(coords)
  
        
        cv2.imshow('live_detection',cv2.resize(image_np,(800,600)))
        
        if coords:
            if coords[0][0]=="person":
                x1=int(coords[0][1])
                y1=int(coords[0][2])
                x2=int(coords[0][3])
                y2=int(coords[0][4])
                print(coords[0][1])
                
                image = cv2.cvtColor(np.array(ImageGrab.grab(bbox=(x1, y1, x2, y2))), cv2.COLOR_BGR2RGB)
                cv2.imshow('person',cv2.resize(image,(800,600)))
        if cv2.waitKey(1)  &0xFF==ord('q'):
            cv2.destroyAllWindows()
            break
    
        #img=ImageGrab.grab((x1,y1,x2,y2))
        

0.004332125186920166


OSError: Pillow was built without XCB support